In [1]:
import requests
from bs4 import BeautifulSoup
# import lxml.html as lh
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import numpy as np
import re
import time
import json

from bokeh.plotting import figure, output_notebook, show

from bokeh.palettes import Reds6,Blues8

In [2]:
Reds6

('#a50f15', '#de2d26', '#fb6a4a', '#fc9272', '#fcbba1', '#fee5d9')

# Schedule

In [3]:
def get_schedule_page(round_num,leagueID=401,
                 url=r"http://sportsdata.misports.cn/beitai/cba/schedule/getschedulelist",
                headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"},
                encoding='utf-8'):
    if round_num >46:
        raise ValueError('CBA共有46轮，请输入小于等于46的数字。')
        return
    if round_num < 0:
        raise ValueError('起始轮必须大于等于0')
        return
    # Get this round's page
    data = {"round": round_num,"leagueID": leagueID}
    r = requests.post(url, data=data, headers=headers)
    if r.status_code != 200:
        print('网页数据返回异常。')
        return
    r.encoding=encoding
    soup = BeautifulSoup(r.content, "html.parser")
    return soup, round_num

In [18]:
# testing 季后赛
headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
data = {"currentdate": '2020-08-04',"leagueID": 401}

r = requests.post(r"http://sportsdata.misports.cn/beitai/cba/schedule/getschedulelistForDate",
              data=data, headers=headers)

r.encoding = 'utf-8'

soup = BeautifulSoup(r.content, "html.parser")

soup

<meta charset="utf-8"/>
<!-- 一级导航结束 -->
<div class="section2">
<div class="time1">08月04日 , 
		    			周二		    </div>
<div class="title">
<span class="stage">阶段</span>
<span class="time">时间</span>
<span class="state">状态</span>
<span class="guest">主队</span>
<span class="score">比分</span>
<span class="main">客队</span>
<span class="lively">直播/回放</span>
</div>
<table border="0" cellpadding="0" cellspacing="0">
<tbody>
<tr class="no">
<td width="156">
					                		季后赛
				                	</td>
<td class="time2" width="130">20:00</td>
<td class="state" width="146">
							                    		未开始
							                    </td>
<td class="guest" width="251"><a href="//sportsdata.misports.cn/beitai/cba/team?leagueid=401&amp;teamid=29124" target="_blank"><span>广东东莞银行</span><img alt="" height="56" src="//sportsdata.misports.cn/collectdata/images/teamimgs/29124.png" width="53"/></a></td>
<td class="score" width="118">
														vs
													
												</td>
<td class="main" wid

In [4]:
def get_games_from_page(soup,round_num):
    current_round_games=[]
    current_round = {}
    current_round['轮次'] = str(round_num)
    # 一个section在网页上对应一个比赛日
    for section in soup.find_all('div',attrs={"class":"section2"}):
        titles = [span.text.strip() for span in section.find('div', attrs={"class":'title'}).find_all('span')]
        games = []
        # 每一行是一场比赛
        for row in section.find('table').find_all('tr'):
            games.append([td.text.strip()+'|'+td.find('a')['href'] if td.find('a') else td.text.strip() for td in row.find_all('td')])
        for game in games:
            if '未开始' in game:
                # exit function
                pass
        for game in games:
            game_dict = {}
            game_dict['比赛日'] = re.sub('\s','',section.find('div',attrs={'class':'time1'}).text)
            for title,data_item in zip(titles,game):
                game_dict[title]=data_item
            current_round_games.append(game_dict)
    
    current_round['本轮比赛'] = current_round_games
    return current_round

# Game Plays

In [5]:
def get_plays(scheduleid,leagueid,
                    headers={
                        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"
                    },
                   url = r"http://sportsdata.misports.cn/beitai/cba/live/liveeventdetails",
                   encoding='UTF-8'):
    data = {"scheduleid": scheduleid,"leagueid": leagueid}
    r = requests.post(url, data=data, headers=headers,)
    r.encoding=encoding
    soup = BeautifulSoup(r.content, "html.parser")
    all_events = []
    for event in soup.find_all("li"):
        event_dic = event.attrs
        li = list(event.find_all('span'))
        if li:
            event_dic['event']=li[2].text
            event_dic['score']=li[3].text
            all_events.append(event_dic)
            
    return all_events

# Scraping

In [ ]:
all_rounds = []
start_round=1
end_round=29
for round_num in range(start_round,end_round+1):
    soup, _ = get_schedule_page(round_num)
    current_round = get_games_from_page(soup,round_num)
    all_rounds.append(current_round)

In [88]:
all_plays=[]
for one_round in all_rounds:
    round_num = one_round['轮次']
    games_this_round = one_round['本轮比赛']
    for game in games_this_round:
        game_date = game['比赛日']
        home_tm = {game['主队'][-5:]:game['主队'].split('|')[0]}
        away_tm = {game['客队'][-5:]:game['客队'].split('|')[0]}
        if game['状态'] =='未开始':
            print(f'Round {round_num}, {game_date}, {home_tm} vs. {away_tm} 未开始，跳过')
            continue
        leagueid = game['比分'].split('&')[0][-3:]
        scheduleid = game['比分'].split('&')[1][-9:]
        print(f'Scraping round {round_num}, {game_date}, {home_tm} vs. {away_tm}, {scheduleid}')
        # game['plays'] = get_events(scheduleid,leagueid)
        all_plays.extend(get_events(scheduleid,leagueid))
        time.sleep(np.random.rand()*3)
        print('-----------------------------')
print('Complete!')

Scraping round 1, 11月01日,周五, {'29124': '广东东莞银行'} vs. {'29129': '辽宁本钢'}, 100025839
-----------------------------
Scraping round 1, 11月02日,周六, {'29118': '苏州肯帝亚'} vs. {'29140': '浙江稠州银行'}, 100025840
-----------------------------
Scraping round 1, 11月02日,周六, {'29127': '四川五粮金樽'} vs. {'29117': '新疆伊力特'}, 100025841
-----------------------------
Scraping round 1, 11月02日,周六, {'29130': '山东西王'} vs. {'29116': '八一南昌'}, 100025849
-----------------------------
Scraping round 1, 11月02日,周六, {'29136': '北京控股'} vs. {'29131': '深圳马可波罗'}, 100025851
-----------------------------
Scraping round 1, 11月02日,周六, {'29125': '上海久事'} vs. {'29133': '南京同曦宙光'}, 100025855
-----------------------------
Scraping round 1, 11月02日,周六, {'29135': '青岛国信双星'} vs. {'29137': '九台农商银行'}, 100025857
-----------------------------
Scraping round 1, 11月02日,周六, {'29128': '浙江广厦控股'} vs. {'29134': '福建豹发力'}, 100025859
-----------------------------
Scraping round 1, 11月03日,周日, {'29138': '天津先行者'} vs. {'29115': '北京首钢'}, 100025861
--------------------

-----------------------------
Scraping round 8, 11月21日,周四, {'29138': '天津先行者'} vs. {'29124': '广东东莞银行'}, 100026117
-----------------------------
Scraping round 8, 11月22日,周五, {'29139': '时代中国广州'} vs. {'29117': '新疆伊力特'}, 100026121
-----------------------------
Scraping round 8, 11月22日,周五, {'29131': '深圳马可波罗'} vs. {'29116': '八一南昌'}, 100026123
-----------------------------
Scraping round 8, 11月22日,周五, {'29133': '南京同曦宙光'} vs. {'29132': '山西汾酒股份'}, 100026125
-----------------------------
Scraping round 8, 11月22日,周五, {'29129': '辽宁本钢'} vs. {'29115': '北京首钢'}, 100026128
-----------------------------
Scraping round 8, 11月22日,周五, {'29137': '九台农商银行'} vs. {'29136': '北京控股'}, 100026130
-----------------------------
Scraping round 9, 11月23日,周六, {'29118': '苏州肯帝亚'} vs. {'29130': '山东西王'}, 100026135
-----------------------------
Scraping round 9, 11月23日,周六, {'29128': '浙江广厦控股'} vs. {'29140': '浙江稠州银行'}, 100026139
-----------------------------
Scraping round 9, 11月23日,周六, {'29135': '青岛国信双星'} vs. {'29127': '四川五粮金樽'

-----------------------------
Scraping round 15, 12月10日,周二, {'29132': '山西汾酒股份'} vs. {'29117': '新疆伊力特'}, 100025973
-----------------------------
Scraping round 15, 12月10日,周二, {'29135': '青岛国信双星'} vs. {'29131': '深圳马可波罗'}, 100025975
-----------------------------
Scraping round 15, 12月11日,周三, {'29127': '四川五粮金樽'} vs. {'29138': '天津先行者'}, 100025978
-----------------------------
Scraping round 16, 12月12日,周四, {'29136': '北京控股'} vs. {'29139': '时代中国广州'}, 100025980
-----------------------------
Scraping round 16, 12月12日,周四, {'29128': '浙江广厦控股'} vs. {'29116': '八一南昌'}, 100025981
-----------------------------
Scraping round 16, 12月12日,周四, {'29129': '辽宁本钢'} vs. {'29125': '上海久事'}, 100025982
-----------------------------
Scraping round 16, 12月12日,周四, {'29137': '九台农商银行'} vs. {'29133': '南京同曦宙光'}, 100025983
-----------------------------
Scraping round 16, 12月12日,周四, {'29134': '福建豹发力'} vs. {'29115': '北京首钢'}, 100025985
-----------------------------
Scraping round 16, 12月12日,周四, {'29124': '广东东莞银行'} vs. {'29140':

-----------------------------
Scraping round 23, 12月31日,周二, {'29127': '四川五粮金樽'} vs. {'29115': '北京首钢'}, 100026027
-----------------------------
Scraping round 23, 12月31日,周二, {'29138': '天津先行者'} vs. {'29117': '新疆伊力特'}, 100026035
-----------------------------
Scraping round 23, 12月31日,周二, {'29130': '山东西王'} vs. {'29134': '福建豹发力'}, 100026039
-----------------------------
Scraping round 23, 12月31日,周二, {'29136': '北京控股'} vs. {'29132': '山西汾酒股份'}, 100026046
-----------------------------
Scraping round 23, 12月31日,周二, {'29139': '时代中国广州'} vs. {'29137': '九台农商银行'}, 100026055
-----------------------------
Scraping round 23, 12月31日,周二, {'29128': '浙江广厦控股'} vs. {'29133': '南京同曦宙光'}, 100026060
-----------------------------
Scraping round 23, 12月31日,周二, {'29135': '青岛国信双星'} vs. {'29116': '八一南昌'}, 100026069
-----------------------------
Scraping round 23, 01月01日,周三, {'29118': '苏州肯帝亚'} vs. {'29129': '辽宁本钢'}, 100026077
-----------------------------
Scraping round 23, 01月01日,周三, {'29125': '上海久事'} vs. {'29140': '浙

In [89]:
df_all_events =pd.DataFrame(all_plays)
df_all_events

,eventid,teamid,shotcoordx,shotcoordy,eventtypeid,quarternum,minutes,seconds,class,quarter,style,event,score,shotevent
0,,0,0,0,19,4,0,0,"[quarterSet, quarter_4]",4,width: 500px;,比赛结束,107:98,NaN
1,,0,0,0,15,4,0,0,"[quarterSet, quarter_4]",4,width: 500px;,第4节结束,107:98,NaN
2,,29129,14.9722,15.54,4,4,0,1,"[quarterSet, quarter_4]",4,width: 500px;,史蒂芬森三分跳投不中，易建联获得防守篮板,107:98,
3,,29124,3.3272,-0.78,3,4,0,10,"[quarterSet, quarter_4]",4,width: 500px;,任骏飞接到易建联的助攻，跳投，命中得分,107:98,
4,,29129,-1.5123,21.78,4,4,0,29,"[quarterSet, quarter_4]",4,width: 500px;,高诗岩三分跳投不中，广东东莞银行队获得防守篮板,105:98,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100725,,29135,0,0,7,1,11,10,"[quarterSet, quarter_1]",1,width: 500px;,许家晗失误,2:0,NaN
100726,,29134,-2.7222,-1.3462,3,1,11,28,"[quarterSet, quarter_1]",1,width: 500px;,王哲林跳投，命中得分,2:0,
100727,,29135,13.6111,19.0385,4,1,11,43,"[quarterSet, quarter_1]",1,width: 500px;,亚当斯三分跳投不中，福建豹发力队获得防守篮板,0:0,
100728,,29134,0,0,12,1,12,0,"[quarterSet, quarter_1]",1,width: 500px;,王哲林和刘传兴跳球，青岛国信双星队得到球权,0:0,NaN


In [7]:
df_all_events.sort_values(by=['eventtypeid']).to_csv('AllPlays_2020-01-20_until_round29.csv',index=False)

# Drawing

In [91]:
df_all_events.columns

Index(['eventid', 'teamid', 'shotcoordx', 'shotcoordy', 'eventtypeid',
       'quarternum', 'minutes', 'seconds', 'class', 'quarter', 'style',
       'event', 'score', 'shotevent'],
      dtype='object')

In [63]:
shooting_events = df_all_events[['teamid','shotcoordx','shotcoordy',
                                 'eventtypeid','quarternum', 'minutes', 'seconds',
                                'event', 'score']].copy()

In [64]:
shooting_events = shooting_events.loc[shooting_events['eventtypeid'].isin(['3','4']),:].copy()

In [65]:
shooting_events['shotcoordx'] = shooting_events['shotcoordx'].astype('float')
shooting_events['shotcoordy'] = shooting_events['shotcoordy'].astype('float')

In [66]:
def meter_to_coord(meter):
    return meter*49/15

In [67]:
xj_shots = shooting_events.query("teamid=='29117'").copy()

In [68]:
xj_shots['color'] = Blues8[1]

In [69]:
xj_shots['fill_color'] = xj_shots['color']

In [70]:
xj_shots.loc[xj_shots['eventtypeid']==4,'fill_color']='white'

In [71]:
gd_shots = shooting_events.query("teamid=='29124'").copy()
gd_shots['color'] = Blues8[1]
gd_shots['fill_color'] = gd_shots['color']
# 没中则用白色填充
gd_shots.loc[gd_shots['eventtypeid']==4,'fill_color']='white'

In [72]:
xj_shots.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2390 entries, 32550 to 78535
Data columns (total 11 columns):
teamid         2390 non-null int64
shotcoordx     2390 non-null float64
shotcoordy     2390 non-null float64
eventtypeid    2390 non-null int64
quarternum     2390 non-null int64
minutes        2390 non-null int64
seconds        2390 non-null int64
event          2390 non-null object
score          2390 non-null object
color          2390 non-null object
fill_color     2390 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 224.1+ KB


In [116]:
print(f'广东{1554+925}投{914+369}中,3分投篮占比{925/(925+1554)*100:.2f}%')

广东2479投1283中,3分投篮占比37.31


In [117]:
print(f'新疆{1496+895}投{873+316}中,3分投篮占比{895/(1496+895)*100:.2f}%')

新疆2391投1189中,3分投篮占比37.43%


In [118]:
print(f'联盟{1496+895}投{873+316}中,3分投篮占比{16340/(30035+16340)*100:.2f}%')

联盟2391投1189中,3分投篮占比35.23%


### 球队分布

In [311]:
import math

In [312]:
def angle(vector1, vector2):
    x1, y1 = vector1
    x2, y2 = vector2
    inner_product = x1*x2 + y1*y2
    len1 = math.hypot(x1, y1)
    len2 = math.hypot(x2, y2)
    return math.acos(inner_product/(len1*len2))

In [315]:
angle((1,1),(-1,0))/pi*180

135.0

In [24]:
endline_y = -7.5
halfcourtline_y=45.73
sideline_left = 24.5
rim_to_baseline=7.5
rim_x=0
rim_y=endline_y+meter_to_coord(1.2)
threepointline_radius = meter_to_coord(6.75)
nochargeline_rarius = meter_to_coord(1.25)
centercircle_radius = meter_to_coord(1.83)
rim_radius = meter_to_coord(0.23)
backboard_length = meter_to_coord(1.83)
restrictedarea_length = meter_to_coord(5.8)
restrictedarea_width = meter_to_coord(4.9)
ftline_length = meter_to_coord(3.6)


goal_color = Blues8[1]
miss_color = Reds6[0]
linewidth = 3
size = 9
alpha = 0.6

In [25]:
def draw_court(p):
    endline_y = -7.5
    halfcourtline_y=45.73
    sideline_left = 24.5
    rim_to_baseline=7.5
    rim_x=0
    rim_y=endline_y+meter_to_coord(1.2)
    threepointline_radius = meter_to_coord(6.75)
    nochargeline_rarius = meter_to_coord(1.25)
    centercircle_radius = meter_to_coord(1.83)
    rim_radius = meter_to_coord(0.23)
    backboard_length = meter_to_coord(1.83)
    restrictedarea_length = meter_to_coord(5.8)
    restrictedarea_width = meter_to_coord(4.9)
    ftline_length = meter_to_coord(3.6)


    goal_color = Blues8[1]
    miss_color = Reds6[0]
    linewidth = 3
    size = 9
    alpha = 0.6
    
    # 3p line
    p.arc(x=rim_x, y=rim_y, radius=threepointline_radius, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)

    # 左侧三分直线
    p.line(x=[-threepointline_radius,-threepointline_radius],y=[endline_y,rim_y],line_color='black', line_width=linewidth-0.1)

    # 右侧三分直线
    p.line(x=[threepointline_radius,threepointline_radius],y=[endline_y,rim_y],line_color='black', line_width=linewidth-0.1)

    # 合理冲撞区
    p.arc(x=rim_x, y=rim_y, radius=nochargeline_rarius, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)

    # 中圈
    p.arc(x=rim_x, y=halfcourtline_y, radius=centercircle_radius, start_angle=pi, end_angle=0,line_color='black', line_width=linewidth)

    # 篮筐
    p.circle(x=0,y=rim_y,radius = rim_radius,color='red',fill_color=None,line_width=linewidth)


    # 篮板
    p.line(x=[-backboard_length/2,backboard_length/2],
           y=[rim_y-rim_radius-0.3,rim_y-rim_radius-0.3],
           line_color='black', line_width=linewidth)

    #场地边线
    p.quad(-sideline_left,sideline_left,halfcourtline_y,endline_y,fill_color=None,line_color='black',
          line_width=linewidth)

    #限制区
    p.quad(-restrictedarea_width/2,restrictedarea_width/2,endline_y+restrictedarea_length,endline_y,fill_color=None,line_color='black',
          line_width=linewidth)

    #罚球区
    p.arc(x=rim_x, y=endline_y+restrictedarea_length, radius=ftline_length/2, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)


In [75]:
from bokeh.models.glyphs import Arc
from math import pi

output_notebook()

# create a new plot
p = figure(
   title="广东19-20赛季投篮分布--2479投1283中，3分投篮占比37.31%",tools="pan,box_zoom,wheel_zoom,save,reset,hover",
    x_range=[-sideline_left,sideline_left],y_range = [endline_y,halfcourtline_y],x_axis_label='x',y_axis_label='y'
)


# p.axis.visible = False
# p.grid.visible = False

# misses
p.cross(gd_shots.loc[gd_shots['eventtypeid']==4,'shotcoordx'],gd_shots.loc[gd_shots['eventtypeid']==4,'shotcoordy'],
        angle=90,color=miss_color, size=size,alpha=alpha,legend_label='未命中')

# goals
p.circle(gd_shots.loc[gd_shots['eventtypeid']==3,'shotcoordx'], gd_shots.loc[gd_shots['eventtypeid']==3,'shotcoordy'], 
         fill_color=goal_color,color=goal_color, size=size,alpha=alpha,legend_label='命中')

draw_court(p)


# show the results
show(p)

#width: 15m, 50
#length: 28m，


Loading BokehJS ...

# Testing Hexagonal Tiles

In [83]:
from bokeh.io import reset_output
reset_output()

In [114]:
import numpy as np

from bokeh.io import output_file, show, reset_output
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
from bokeh.palettes import inferno

output_notebook()

n = 50000
x = gd_shots.loc[gd_shots['eventtypeid']==3,'shotcoordx'].values
y = gd_shots.loc[gd_shots['eventtypeid']==3,'shotcoordy'].values
color_grade = inferno(50)
color_grade.reverse()

bins = hexbin(x, y, 1)

p = figure(
   title="广东19-20赛季投篮分布--2479投1283中，3分投篮占比37.31%",tools="save,reset",
    x_range=[-sideline_left,sideline_left],y_range = [endline_y,halfcourtline_y],x_axis_label='x',y_axis_label='y',
    match_aspect=True, background_fill_color=None
)

p.grid.visible = False

p.hex_tile(q="q", r="r", size=1, line_color=None, source=bins,
           fill_color=linear_cmap('counts', color_grade, 0, max(bins.counts)),alpha=0.8)
draw_court(p)

show(p)

Loading BokehJS ...

In [99]:
Blues8

AttributeError: 'list' object has no attribute 'invert'

In [378]:

# output to static HTML file
output_notebook()

# create a new plot
p = figure(
   title="新疆19-20赛季投篮分布--2391投1189中，3分投篮占比37.43%",tools="pan,box_zoom,wheel_zoom,save,reset,hover",
    x_range=[-24.5,24.5],y_range = [-7.5,45.73],x_axis_label='xxx',y_axis_label='yyy'
)
p.axis.visible = False

# misses
p.cross(xj_shots.loc[xj_shots['eventtypeid']=='4','shotcoordx'],xj_shots.loc[xj_shots['eventtypeid']=='4','shotcoordy'],
        angle=90,color=miss_color, size=size,alpha=alpha,legend_label='未命中')

# goals
p.circle(xj_shots.loc[xj_shots['eventtypeid']=='3','shotcoordx'], xj_shots.loc[xj_shots['eventtypeid']=='3','shotcoordy'], 
         fill_color=goal_color,color=goal_color, size=size,alpha=alpha,legend_label='命中')

draw_court(p)

# show the results
show(p)

#width: 15m, 50
#length: 28m，


Loading BokehJS ...

### 周琦vs易建联

In [227]:
zq_shots = xj_shots.loc[(xj_shots['event'].str.contains('周琦'))&(-xj_shots['event'].str.contains('周琦抢到'))&(-(xj_shots['event'].str.contains('周琦的助攻'))),:].copy()


In [226]:
yjl_shots = gd_shots.loc[(gd_shots['event'].str.contains('易建联'))&(-gd_shots['event'].str.contains('易建联抢到'))&(-(gd_shots['event'].str.contains('易建联的助攻'))),:].copy()


In [241]:
print(f'易建联共{296+94}投{168+34}中，总命中率{(168+34)/(296+94)*100:.2f}%，3分投篮占比{94/(296+94)*100:.2f}%')

易建联共390投202中，总命中率51.79%，3分投篮占比24.10%


In [240]:
print(f'周琦共{243+103}投{33+157}中，总命中率{(33+157)/(243+103)*100:.2f}%，3分投篮占比{103/(243+103)*100:.2f}%')

周琦共346投190中，总命中率54.91%，3分投篮占比29.77%


In [379]:
from bokeh.models.glyphs import Arc
from math import pi

# prepare some data

# output to static HTML file
output_notebook()

# create a new plot
p = figure(
   title="周琦19-20赛季投篮分布--共346投190中，总命中率54.91%，3分投篮占比29.77%",tools="pan,box_zoom,wheel_zoom,save,reset,hover",
    x_range=[-24.5,24.5],y_range = [-7.5,45.73],x_axis_label='x',y_axis_label='y'
)

p.axis.visible = False

# misses
p.cross(zq_shots.loc[zq_shots['eventtypeid']=='4','shotcoordx'],zq_shots.loc[zq_shots['eventtypeid']=='4','shotcoordy'],
        angle=90,color=miss_color, size=10,alpha=alpha,legend_label='未命中')

# goals
p.circle(zq_shots.loc[zq_shots['eventtypeid']=='3','shotcoordx'], zq_shots.loc[zq_shots['eventtypeid']=='3','shotcoordy'], 
         fill_color=goal_color,color=goal_color, size=10,alpha=alpha,legend_label='命中')

draw_court(p)

# show the results
show(p)

#width: 15m, 50
#length: 28m，


Loading BokehJS ...

In [380]:
# prepare some data

# output to static HTML file
output_notebook()

# create a new plot
p = figure(
   title="易建联19-20赛季投篮分布--共390投202中，总命中率51.79%，3分投篮占比24.10%",tools="pan,box_zoom,wheel_zoom,save,reset,hover",
    x_range=[-24.5,24.5],y_range = [-7.5,45.73],x_axis_label='xxx',y_axis_label='yyy'
)
p.axis.visible = False

# misses
p.cross(yjl_shots.loc[yjl_shots['eventtypeid']=='4','shotcoordx'],yjl_shots.loc[yjl_shots['eventtypeid']=='4','shotcoordy'],
        angle=90,color=miss_color, size=10,alpha=alpha,legend_label='未命中')

# goals
p.circle(yjl_shots.loc[yjl_shots['eventtypeid']=='3','shotcoordx'], yjl_shots.loc[yjl_shots['eventtypeid']=='3','shotcoordy'], 
         fill_color=goal_color,color=goal_color, size=10,alpha=alpha,legend_label='命中')

draw_court(p)

# show the results
show(p)

#width: 15m, 50
#length: 28m，


Loading BokehJS ...